In [6]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score

BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
FOURTH_MOD_DIR = "/home/abhinav/data/fourth_model/"

print('Loading data...')
[x_train, y_train] = pd.read_pickle(FOURTH_MOD_DIR + "train_data_10ratio") 

[x_test, y_test] = pd.read_pickle(FOURTH_MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(FOURTH_MOD_DIR + "validation_dataset")

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
num_words = MAX_NB_WORDS

embedding_matrix = pd.read_pickle("/home/abhinav/data/fourth_model/embedding_matrix_for_selected_words")

Loading data...


In [2]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("\nWeighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array

In [7]:

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


In [17]:
del(model)


In [22]:
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(10)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 20)                16880     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 42        
Total params: 10,016,922
Trainable params: 16,922
Non-trainable params: 10,000,000
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

In [25]:
initial_weights = model.get_weights()

In [17]:
del(model)


model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(10)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

In [26]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.2553 - acc: 0.5450
Weighted F1 score found on Validation dataset :  (array([ 0.99346313,  0.07675906]), array([ 0.94226667,  0.43636364]), array([ 0.96718787,  0.13055304]), array([15000,   165]))
13150/13150 [==============================] - 302s - loss: 1.2535 - acc: 0.5468 - val_loss: 0.4292 - val_acc: 0.9368
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0710 - acc: 0.7330
Weighted F1 score found on Validation dataset :  (array([ 0.99796679,  0.0419518 ]), array([ 0.78533333,  0.85454545]), array([ 0.87897329,  0.07997731]), array([15000,   165]))
13150/13150 [==============================] - 283s - loss: 1.0697 - acc: 0.7325 - val_loss: 0.5510 - val_acc: 0.7861
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.9197 - acc: 0.8258
Weighted F1 score found on Validation dataset :  (array([ 0.99750254,  0.

In [27]:
del(model)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(20)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_7 (Bidirection (None, 40)                35360     
_________________________________________________________________
dropout_7 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 82        
Total params: 10,035,442
Trainable params: 35,442
Non-trainable params: 10,000,000
_________________________________________________________________


In [28]:

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

In [29]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.2686 - acc: 0.5305
Weighted F1 score found on Validation dataset :  (array([ 0.99825175,  0.02477964]), array([ 0.60906667,  0.9030303 ]), array([ 0.7565419 ,  0.04823567]), array([15000,   165]))
13150/13150 [==============================] - 357s - loss: 1.2676 - acc: 0.5313 - val_loss: 0.7179 - val_acc: 0.6123
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.0139 - acc: 0.7014
Weighted F1 score found on Validation dataset :  (array([ 0.99917432,  0.02867056]), array([ 0.6454    ,  0.95151515]), array([ 0.78423589,  0.05566389]), array([15000,   165]))
13150/13150 [==============================] - 356s - loss: 1.0164 - acc: 0.7013 - val_loss: 0.6159 - val_acc: 0.6487
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.0529 - acc: 0.7223
Weighted F1 score found on Validation dataset :  (array([ 0.99829802,  0.

In [30]:
del(model)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(30)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_8 (Bidirection (None, 60)                55440     
_________________________________________________________________
dropout_8 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 122       
Total params: 10,055,562
Trainable params: 55,562
Non-trainable params: 10,000,000
_________________________________________________________________


In [31]:
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.2763 - acc: 0.5035
Weighted F1 score found on Validation dataset :  (array([ 0.9979327 ,  0.02276247]), array([ 0.57926667,  0.89090909]), array([ 0.73303244,  0.04439076]), array([15000,   165]))
13150/13150 [==============================] - 472s - loss: 1.2746 - acc: 0.5043 - val_loss: 0.6256 - val_acc: 0.5827
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.0156 - acc: 0.7341
Weighted F1 score found on Validation dataset :  (array([ 0.99797796,  0.02749336]), array([ 0.65806667,  0.87878788]), array([ 0.79313808,  0.05331862]), array([15000,   165]))
13150/13150 [==============================] - 470s - loss: 1.0149 - acc: 0.7332 - val_loss: 0.5930 - val_acc: 0.6605
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 0.9301 - acc: 0.7567
Weighted F1 score found on Validation dataset :  (array([ 0.99782135,  0.

In [32]:
del(model)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 100)               100400    
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 202       
Total params: 10,100,602
Trainable params: 100,602
Non-trainable params: 10,000,000
_________________________________________________________________


In [33]:

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 2s - loss: 1.3247 - acc: 0.5064
Weighted F1 score found on Validation dataset :  (array([ 0.99459327,  0.0172193 ]), array([ 0.5396    ,  0.73333333]), array([ 0.69962832,  0.0336485 ]), array([15000,   165]))
13150/13150 [==============================] - 702s - loss: 1.3231 - acc: 0.5059 - val_loss: 0.6432 - val_acc: 0.5417
Epoch 2/20
13056/13150 [============================>.] - ETA: 2s - loss: 1.0625 - acc: 0.7044
Weighted F1 score found on Validation dataset :  (array([ 0.99833038,  0.02669294]), array([ 0.6378   ,  0.9030303]), array([ 0.77834276,  0.05185314]), array([15000,   165]))
13150/13150 [==============================] - 701s - loss: 1.0616 - acc: 0.7051 - val_loss: 0.8109 - val_acc: 0.6407
Epoch 3/20
13056/13150 [============================>.] - ETA: 2s - loss: 0.8392 - acc: 0.7948
Weighted F1 score found on Validation dataset :  (array([ 0.99785383,  0.02

In [ ]:
del(model)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 200)               240800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 402       
Total params: 10,241,202
Trainable params: 241,202
Non-trainable params: 10,000,000
_________________________________________________________________


In [ ]:

model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 5s - loss: 1.2366 - acc: 0.5598 
Weighted F1 score found on Validation dataset :  (array([ 0.9985158 ,  0.03988143]), array([ 0.76246667,  0.8969697 ]), array([ 0.86467075,  0.07636739]), array([15000,   165]))
13150/13150 [==============================] - 1390s - loss: 1.2362 - acc: 0.5615 - val_loss: 0.4625 - val_acc: 0.7639
Epoch 2/20
13056/13150 [============================>.] - ETA: 5s - loss: 0.9493 - acc: 0.7894 
Weighted F1 score found on Validation dataset :  (array([ 0.99784826,  0.05273214]), array([ 0.83473333,  0.83636364]), array([ 0.90903151,  0.0992092 ]), array([15000,   165]))
13150/13150 [==============================] - 1387s - loss: 0.9469 - acc: 0.7897 - val_loss: 0.4674 - val_acc: 0.8348
Epoch 3/20
13056/13150 [============================>.] - ETA: 5s - loss: 0.8204 - acc: 0.8098 
Weighted F1 score found on Validation dataset :  (array([ 0.9973262 

In [ ]:
del(model)

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH,trainable = False))
model.add(Bidirectional(LSTM(150)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.summary()


In [ ]:


model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])

initial_weights = model.get_weights()

model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 12},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 20},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [14]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 30},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.5838 - acc: 0.4036
Weighted F1 score found on Validation dataset :  (array([ 0.99924573,  0.0482524 ]), array([ 0.79486667,  0.94545455]), array([ 0.88541512,  0.09181872]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 1.5791 - acc: 0.4064 - val_loss: 0.5419 - val_acc: 0.7965
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.9673 - acc: 0.7640
Weighted F1 score found on Validation dataset :  (array([ 0.99930957,  0.04387926]), array([ 0.77193333,  0.95151515]), array([ 0.87102719,  0.08388993]), array([15000,   165]))
13150/13150 [==============================] - 320s - loss: 0.9671 - acc: 0.7647 - val_loss: 0.5668 - val_acc: 0.7739
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6484 - acc: 0.8555
Weighted F1 score found on Validation dataset :  (array([ 0.99766679,  0.

In [10]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 10},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 1s - loss: 1.0621 - acc: 0.6901
Weighted F1 score found on Validation dataset :  (array([ 0.9995951 ,  0.03045781]), array([ 0.65833333,  0.97575758]), array([ 0.7938422 ,  0.05907173]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 1.0628 - acc: 0.6911 - val_loss: 0.6460 - val_acc: 0.6618
Epoch 2/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.6110 - acc: 0.8635
Weighted F1 score found on Validation dataset :  (array([ 0.99835858,  0.08115778]), array([ 0.89206667,  0.86666667]), array([ 0.94222441,  0.14841723]), array([15000,   165]))
13150/13150 [==============================] - 327s - loss: 0.6098 - acc: 0.8636 - val_loss: 0.2627 - val_acc: 0.8918
Epoch 3/10
13056/13150 [============================>.] - ETA: 1s - loss: 0.4403 - acc: 0.8929
Weighted F1 score found on Validation dataset :  (array([ 0.99821681,  0.

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 50},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=15,
          class_weight = {0. : 1, 1. : 75},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 100},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [9]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/10
13056/13150 [============================>.] - ETA: 2s - loss: 0.9033 - acc: 0.8074
Weighted F1 score found on Validation dataset :  (array([ 0.99944721,  0.03688239]), array([ 0.7232    ,  0.96363636]), array([ 0.83917382,  0.07104558]), array([15000,   165]))
13150/13150 [==============================] - 648s - loss: 0.9021 - acc: 0.8077 - val_loss: 0.5441 - val_acc: 0.7258
Epoch 2/10
13056/13150 [============================>.] - ETA: 2s - loss: 0.5040 - acc: 0.8678
Weighted F1 score found on Validation dataset :  (array([ 0.99735884,  0.11072664]), array([ 0.93146667,  0.77575758]), array([ 0.96328726,  0.19379258]), array([15000,   165]))
13150/13150 [==============================] - 655s - loss: 0.5026 - acc: 0.8682 - val_loss: 0.1606 - val_acc: 0.9298
Epoch 3/10
13056/13150 [============================>.] - ETA: 2s - loss: 0.3329 - acc: 0.9126
Weighted F1 score found on Validation dataset :  (array([ 0.99651228,  0.

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 125},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [17]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 150},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 2.7308 - acc: 0.1568
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01704545]), array([ 0.36566667,  1.        ]), array([ 0.53551379,  0.03351955]), array([15000,   165]))
13150/13150 [==============================] - 321s - loss: 2.7281 - acc: 0.1587 - val_loss: 1.2697 - val_acc: 0.3726
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.8886 - acc: 0.5123
Weighted F1 score found on Validation dataset :  (array([ 0.99960684,  0.03225806]), array([ 0.678     ,  0.97575758]), array([ 0.80797648,  0.06245151]), array([15000,   165]))
13150/13150 [==============================] - 319s - loss: 1.8832 - acc: 0.5138 - val_loss: 0.8559 - val_acc: 0.6812
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.3161 - acc: 0.7084
Weighted F1 score found on Validation dataset :  (array([ 0.9993245 ,  0.

KeyboardInterrupt: 

In [16]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 200},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 3.3969 - acc: 0.0895
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 328s - loss: 3.3978 - acc: 0.0898 - val_loss: 2.6060 - val_acc: 0.0109
Epoch 2/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.9941 - acc: 0.3591
Weighted F1 score found on Validation dataset :  (array([ 1.        ,  0.01807427]), array([ 0.4024,  1.    ]), array([ 0.57387336,  0.03550678]), array([15000,   165]))
13150/13150 [==============================] - 331s - loss: 1.9972 - acc: 0.3592 - val_loss: 1.7261 - val_acc: 0.4089
Epoch 3/20
13056/13150 [============================>.] - ETA: 1s - loss: 1.2913 - acc: 0.7218
Weighted F1 score found on Validation dataset :  (array([ 0.9995438 ,  0.02516805]), array([ 0.58

In [ ]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 500},
validation_data=[x_test, y_test],
         callbacks = [metrics])

In [15]:
model.set_weights(initial_weights)
metrics = Metrics(FOURTH_MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          class_weight = {0. : 1, 1. : 1000},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train on 13150 samples, validate on 15165 samples
Epoch 1/20
13056/13150 [============================>.] - ETA: 1s - loss: 8.1149 - acc: 0.0897
Weighted F1 score found on Validation dataset :  (array([ 0.        ,  0.01088032]), array([ 0.,  1.]), array([ 0.        ,  0.02152642]), array([15000,   165]))
13150/13150 [==============================] - 325s - loss: 8.0891 - acc: 0.0895 - val_loss: 3.4302 - val_acc: 0.0109

/home/abhinav/miniconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 2/20
 7168/13150 [===============>..............] - ETA: 86s - loss: 3.4647 - acc: 0.0871

KeyboardInterrupt: 